In [ ]:
import numpy as np
import scipy.sparse
import numba

In [ ]:
data = np.random.random((2*16384, 2*16384)).astype(np.float32)

In [18]:
sparse_mask_left = scipy.sparse.csr_matrix(([1.]*1000, (range(1000), [0]*1000)), shape=(2*16384, 1), dtype=np.float32)

In [ ]:
%timeit sparse_mask_left.__rmatmul__(data)

In [ ]:
%timeit sparse_mask_left.__rmatmul__(data.T)

In [ ]:
dense_mask_left = sparse_mask_left.toarray()

In [ ]:
%timeit dense_mask_left.__rmatmul__(data)

In [20]:
sparse_mask_right = scipy.sparse.csr_matrix(([1.]*1000, ([0]*1000, range(1000))), shape=(1, 2*16384), dtype=np.float32)

In [21]:
%timeit sparse_mask_right.__matmul__(data)

13.1 ms ± 46.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit sparse_mask_right.__matmul__(data.T)

In [ ]:
dense_mask_right = sparse_mask_right.toarray()

In [ ]:
%timeit dense_mask_right.__matmul__(data)

In [ ]:
import sys, scipy, numpy; print(scipy.__version__, numpy.__version__, sys.version_info)

In [36]:
test_data = np.array((
    [2, 3, 5],
    [7, 11, 13],
    [1, 2, 3],
    [4, 5, 6]
))

test_sparse = scipy.sparse.csc_matrix(np.array((
    [17, 19],
    [23, 29],
    [31, 37]
)))

In [48]:
test_data @ test_sparse

array([[258, 310],
       [775, 933],
       [156, 188],
       [369, 443]])

In [8]:
test_sparse.indices

array([0, 1, 2, 0, 1, 2], dtype=int32)

In [9]:
test_sparse.indptr

array([0, 3, 6], dtype=int32)

In [10]:
test_sparse.data

array([17, 23, 31, 19, 29, 37])

In [49]:
def rmatmul(left_dense, right_sparse):
    if len(left_dense.shape) != 2:
        raise ValueError(f"Shape of left_dense is not 2D, but {left_dense.shape}")
    if len(right_sparse.shape) != 2:
        raise ValueError(f"Shape of right_sparse is not 2D, but {right_sparse.shape}")
    if left_dense.shape[1] != right_sparse.shape[0]:
        raise ValueError(
            f"Shape mismatch: left_dense.shape[1] != right_sparse.shape[0], "
            f"got {left_dense.shape[1], right_sparse.shape[0]}."
        )
    
    
    result_t = np.zeros(
        shape=(right_sparse.shape[1], left_dense.shape[0]),
        dtype=np.result_type(right_sparse, left_dense)
    )
    
    if isinstance(right_sparse, scipy.sparse.csc_matrix):
        _rmatmul_csc(
            left_dense=left_dense,
            right_data=right_sparse.data,
            right_indices=right_sparse.indices,
            right_indptr=right_sparse.indptr,
            res_inout_t=result_t
        )
    elif isinstance(right_sparse, scipy.sparse.csr_matrix):
        _rmatmul_csr(
            left_dense=left_dense,
            right_data=right_sparse.data,
            right_indices=right_sparse.indices,
            right_indptr=right_sparse.indptr,
            res_inout_t=result_t
        )
    else:
        raise ValueError(
            f"Right hand matrix mus be of type scipy.sparse.csc_matrix or scipy.sparse.csr_matrix, "
            f"got {type(right_sparse)}."
        )
    return result_t.T.copy()


@numba.njit(fastmath=True)
def _rmatmul_csc(left_dense, right_data, right_indices, right_indptr, res_inout_t):
    left_columns = left_dense.shape[1]
    left_rows = left_dense.shape[0]
    for right_column in range(len(right_indptr) - 1):
        offset = right_indptr[right_column]
        items = right_indptr[right_column+1] - offset
        if items > 0:
            for i in range(items):
                index = i + offset
                right_row = right_indices[index]
                right_value = right_data[index]
                for left_row in range(left_rows):
                    tmp = left_dense[left_row, right_row] * right_value
                    res_inout_t[right_column, left_row] += tmp

@numba.njit(fastmath=True)
def _rmatmul_csr(left_dense, right_data, right_indices, right_indptr, res_inout_t):
    left_columns = left_dense.shape[1]
    left_rows = left_dense.shape[0]
    rowbuf = np.empty(shape=(left_rows,), dtype=left_dense.dtype)
    for right_row in range(len(right_indptr) - 1):
        offset = right_indptr[right_row]
        items = right_indptr[right_row+1] - offset
        if items > 0:
            rowbuf[:] = left_dense[:, right_row]
            for i in range(items):
                index = i + offset
                right_column = right_indices[index]
                right_value = right_data[index]
                
                for left_row in range(left_rows):
                    tmp = rowbuf[left_row] * right_value
                    res_inout_t[right_column, left_row] += tmp

In [50]:
rmatmul(test_data, test_sparse.tocsc())

array([[258, 310],
       [775, 933],
       [156, 188],
       [369, 443]])

In [52]:
rmatmul(test_data, test_sparse.tocsr())

array([[258, 310],
       [775, 933],
       [156, 188],
       [369, 443]])

In [53]:
%timeit rmatmul_csc(data, sparse_mask_left.tocsc())

511 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%timeit rmatmul_csr(data, sparse_mask_left.tocsr())

552 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
sparse_mask_left.indptr

In [ ]:
sparse_mask_left

In [ ]:
sparse_mask_left.tocsc().indptr